In [1]:
#Librerias:
import pandas as pd
import numpy as np

## Consideraciones para la implementación:
* Se considera velocidad constante para todos los pickeadores
* Se considera el tiempo en recoger un producto igual para todos los pickeadores y para todos los productos

In [2]:
cant_pickeadores=4 #cantidad de pickeadores en el centro de distribución.
tiempo_pickeo=20 #tiempo de pickeo por producto (se considera constante).
velocidad=30 #velocidad de los pickeadores para recorrer el centro. 

##  Extraccion de datos:

In [3]:
ots = pd.read_csv("data/ot.csv",sep=',')
ots=ots.sort_values('Pedido')
xlsx_file = "data/layout.xlsx"
layout = pd.read_excel(xlsx_file, sheet_name="layout")
adyacencia=pd.read_excel(xlsx_file, sheet_name="adyacencia")


In [4]:
#ordenes:
ots.head()

,Pedido,Cod.Prod
0,1,9.0
1,2,69.0
2,2,42.0
3,2,44.0
4,2,27.0


In [5]:
#datos sobre cada producto/pasillo
layout.head()

,Producto,orden,pasillo,uso,direccion,gondola,slot_largo,slot_ancho,slot_alto,largo_pasillo,ancho_pasillo
0,1,1,11,Pallet Pck General,ASCENDENTE,izquierda,9999,9999,9999,9999,9999
1,2,2,11,Pallet Pck General,ASCENDENTE,derecha,9999,9999,9999,9999,9999
2,3,3,11,Pallet Pck General,ASCENDENTE,izquierda,9999,9999,9999,9999,9999
3,4,4,11,Pallet Pck General,ASCENDENTE,derecha,9999,9999,9999,9999,9999
4,5,5,11,Pallet Pck General,ASCENDENTE,izquierda,9999,9999,9999,9999,9999


In [6]:
#adyacencia de cada pasillo:
adyacencia.head()

,pasillo,adyacente,distancia
0,0,11,1
1,0,31,1
2,0,51,1
3,11,12,1
4,11,21,1


In [7]:
#cantidad de ordenes 
cant_ordenes=len(ots['Pedido'].unique())
cant_ordenes

20

In [8]:
#array con todos los pasillos:
pasillos=layout['pasillo'].unique()
pasillos

array([11, 12, 22, 21, 31, 32, 42, 41, 51, 52, 62, 61], dtype=int64)

In [9]:
#lista con las ordenes ordenadas 
ordenes_enum=ots["Pedido"].unique()
ordenes_enum=list(ordenes_enum)
ordenes_enum

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [10]:
#arreglo con cada producto pedido por orden
lista=[]
for x in range(cant_ordenes):
    obj=list(ots.loc[ots["Pedido"]==x+1]["Cod.Prod"])
    lista.append(obj)
    ordenes=np.array(lista)
ordenes

array([list([9.0]), list([69.0, 42.0, 44.0, 27.0, 37.0, 54.0, 3.0]),
       list([62.0, 68.0, 1.0, 2.0]),
       list([63.0, 16.0, 13.0, 41.0, 66.0, 69.0, 70.0, 72.0]),
       list([67.0, 44.0, 58.0]), list([70.0, 58.0]),
       list([3.0, 55.0, 52.0, 22.0, 58.0, 66.0, 57.0, 5.0]),
       list([48.0, 9.0, 55.0]), list([55.0]), list([8.0, 20.0, 67.0]),
       list([61.0, 57.0]), list([57.0]),
       list([69.0, 34.0, 24.0, 22.0, 59.0, 19.0, 63.0]), list([55.0]),
       list([14.0, 35.0, 17.0, 65.0, 59.0, 72.0, 56.0, 27.0]),
       list([49.0, 13.0, 29.0, 2.0, 50.0]), list([14.0]),
       list([30.0, 33.0, 5.0]), list([35.0, 43.0, 20.0, 58.0, 52.0, 7.0]),
       list([6.0])], dtype=object)

In [11]:
#se construye una lista donde cada elemento representa el pedido asignado a cada pickeador
#(inicialmente 0 porque no tienen pedidos asignados)
pickeadores=[]
for x in range(cant_pickeadores):
    pickeadores.append(0)

pickeadores

[0, 0, 0, 0]

In [12]:
#se inicializa un dataframe que me dira si un pasillo esta ocupado y que pickeador está en el (pickeador 0 cuando no hay nadie).
l=[]
for pasillo in pasillos:
    l.append([pasillo,False, 0])
pasillo_bool=pd.DataFrame(l,
                          columns=["pasillo","ocupado","pickeador"])

pasillo_bool.head()

,pasillo,ocupado,pickeador
0,11,False,0
1,12,False,0
2,22,False,0
3,21,False,0
4,31,False,0


In [13]:
#se crea una lista donde cada elemento en la posicion i representa el pasillo actual del pickeador i+1.
pasillo_act=[]
for x in range(cant_pickeadores):
    pasillo_act.append(0)
    
pasillo_act

[0, 0, 0, 0]

## Funciones:

In [14]:
#Funcion para asignar ordenes a los pickeadores, recibe una lista talcomo la que creamos antes "pickeadores"
def asignar_ordenes(pickeadores):
    for x in range(cant_pickeadores):
        if pickeadores[x]==0:
            if len(ordenes_enum)>0:
                pickeadores[x]=ordenes_enum[0]
                ordenes_enum.remove(ordenes_enum[0])
    return pickeadores

In [15]:
#Funcion que retorna True si es que existe ruta desde el pasillo p2 al p1, y False en caso contrario
def existe_ruta(p1,p2):
    ady=adyacencia.loc[adyacencia["pasillo"]==p2]
    
    if p1==p2:
        return True
    
    for x in range(ady.shape[0]):
        if ady.iloc[x,1]==p1:
            return True
        else:
            if ady.iloc[x,1]==-1:
                return False
            else:
                return existe_ruta(p1,ady.iloc[x,1])

In [16]:
#función que entrega el siguiente pasillo a ir en la ruta desde un pasillo a otro.
def next_move(pasillo_a_ir,pasillo_act):
    ady=adyacencia.loc[adyacencia["pasillo"]==pasillo_act]
    for x in range(ady.shape[0]):
        if existe_ruta(pasillo_a_ir,ady.iloc[x,1]):
            return ady.iloc[x,1]

In [17]:
#Función que me devuelve una lista con la ruta a seguir para ir a un pasillo desde el pasillo actual en el que se encuentra un
#pickeador.
def ruta(pasillo_a_ir,pasillo_act):
    x=pasillo_act
    l=[]
    while x!=pasillo_a_ir:
        x=next_move(pasillo_a_ir,x)
        l.append(x)
    return l

In [18]:
#Funcion creada para ordenar por pasillo los elementos dentro de una orden.
def ordenar(orden):
    lista_ordenada=[]
    for pasillo in pasillos:
        for x in range(len(orden)):
        
            if pasillo==objeto_en_pasillo(orden[x]):
                lista_ordenada.append(orden[x])
    return lista_ordenada

In [19]:
#Funcion que recibe un codigo de producto y retorna el pasillo en el que se encuentra
def objeto_en_pasillo(cod_prod):
    l=layout.loc[layout["Producto"]==cod_prod]
    pasillo=l.iloc[0,2]
    return pasillo

In [20]:
#Funcion que recibe un pasillo y retorna True si es que este esta ocupado o False en el caso contrario.
def ocupado(pasillo):
    p=pasillo_bool.loc[pasillo_bool["pasillo"]==pasillo]
    return p.iloc[0,1]

## Función final:

In [21]:
def tiempo(ordenes,pickeadores):
    print("cant pickeadores:", cant_pickeadores)
    for x in range(len(ordenes)):
        ordenes[x]=ordenar(ordenes[x])
    
    t_recorrido=0
    t_wait=0
    t_pick=0
    contador=[]
    for y in range(cant_pickeadores):
        contador.append(0)
        
    while len(ordenes_enum)>0:
        
        asignar_ordenes(pickeadores)
        for x in range(cant_pickeadores):
            if pickeadores[x] !=0: 
                if contador[x]==-1:
                    next_move=-1
                else:
                    next_move=objeto_en_pasillo(ordenes[pickeadores[x]-1][contador[x]])
                if len(ruta(next_move,pasillo_act[x]))==0:
                    if pasillo_act[x]==-1:
                        contador[x]=0
                        pickeadores[x]=0
                        pasillo_act[x]=0
                    
                    else:
                        t_pick+=tiempo_pickeo
                        if contador[x]+1<len(ordenes[pickeadores[x]-1]):
                            contador[x]+=1
                        else:
                            contador[x]=-1
                else:
                    route=ruta(next_move,pasillo_act[x])
                    if route[0]==-1:
                        pasillo_bool.iloc[pasillo_bool.loc[pasillo_bool["pasillo"]==pasillo_act[x]].index,1]=False
                        pasillo_bool.iloc[pasillo_bool.loc[pasillo_bool["pasillo"]==pasillo_act[x]].index,2]=0
                        t_recorrido+=1/velocidad
                        pasillo_act[x]=route[0]
                        pasillo_bool.iloc[pasillo_bool.loc[pasillo_bool["pasillo"]==route[0]].index,1]=True
                        pasillo_bool.iloc[pasillo_bool.loc[pasillo_bool["pasillo"]==route[0]].index,2]=x+1
                    
                    else:
                        if ocupado(route[0]):
                            t_wait+=tiempo_pickeo
                        else:
                            if pasillo_act[x]>0:
                                largo_pasillo=layout.loc[layout["pasillo"]==pasillo_act[x]].iloc[0,9]
                                t_recorrido+=largo_pasillo/velocidad
                            distancia_df=adyacencia.loc[(adyacencia["pasillo"]==pasillo_act[x]) & (adyacencia["adyacente"]==route[0])]
                            distancia=distancia_df.iloc[0,2]
                            pasillo_bool.iloc[pasillo_bool.loc[pasillo_bool["pasillo"]==pasillo_act[x]].index,1]=False
                            pasillo_bool.iloc[pasillo_bool.loc[pasillo_bool["pasillo"]==pasillo_act[x]].index,2]=0
                            t_recorrido+=distancia/velocidad
                            pasillo_act[x]=route[0]
                            pasillo_bool.iloc[pasillo_bool.loc[pasillo_bool["pasillo"]==route[0]].index,1]=True
                            pasillo_bool.iloc[pasillo_bool.loc[pasillo_bool["pasillo"]==route[0]].index,2]=x+1
            
    print ("tiempo de espera:", t_wait)
    print("tiempo en hacer recorridos:", t_recorrido)
    print("tiempo de pickeo:", t_pick)
    
    return t_recorrido+t_wait+t_pick


In [22]:
tiempo(ordenes,pickeadores)

cant pickeadores: 4
tiempo de espera: 500
tiempo en hacer recorridos: 66001.2333333335
tiempo de pickeo: 1360


67861.2333333335

## Problemas:
* Al momento de escoger la cantidad de pickeadores mayor igual a las ordenes totales (20 en el caso de prueba) surge un error en el que el tiempo de pickeo de productos es 0, lo que nos dice que no recogerian productos los pickeadores.
* La función ruta, no retorna la ruta mas corta entre 2 pasillos, por ejemplo devuelve rutas de 6 pasillos cuando el pasillo esta a 2.

In [24]:
#Notar que el pasillo 51 es adyacente al pasillo 0 entonces solo basta un movimiento para ir, en cambio la funcion retorna:
ruta(51,0)

[11, 12, 22, 21, 31, 32, 42, 41, 51]